In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# 1) Load data (file output sau khi bạn xử lý df)
df = pd.read_csv("hanoi_weather_data.csv")
# basic cleaning: remove unnamed index column if present
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# ensure time column exists and is datetime
if "time" in df.columns:
    df["time"] = pd.to_datetime(df["time"], errors="coerce")

FileNotFoundError: [Errno 2] No such file or directory: 'hanoi_weather_data.csv'

In [ ]:
# 2) tạo AQI category từ PM2.5 (xài rule US EPA)

def pm25_to_aqi_cat(x):
    if x <= 12: return "Good"
    elif x <= 35.4: return "Moderate"
    elif x <= 55.4: return "Unhealthy for SG"
    elif x <= 150.4: return "Unhealthy"
    else: return "Very Unhealthy"

# target
df["AQI_cat"] = df["pm2_5"].apply(pm25_to_aqi_cat)

In [ ]:
# 3) chọn features (pollutants + weather) — dùng danh sách rõ ràng
feature_cols = [
    "pm10","pm2_5","carbon_monoxide","nitrogen_dioxide","ozone","sulphur_dioxide"
]
# giữ lại các cột thực sự có trong data
keep_cols = [c for c in feature_cols if c in df.columns]
# đảm bảo có cả cột AQI_cat
df = df[keep_cols + ["time","AQI_cat"]].copy()

# chuyển đổi sang số và loại bỏ hàng thiếu
df[keep_cols] = df[keep_cols].apply(pd.to_numeric, errors="coerce")
df = df.dropna(subset=["pm2_5"])  # cần pm2_5 để tạo nhãn
df = df.dropna(subset=keep_cols)

# chuẩn bị X, y
X = df[keep_cols].values
y = df["AQI_cat"].values

# encode label
le = LabelEncoder()
y = le.fit_transform(y)

# scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 4) tạo model MLP
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 5) train
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/30
191/191 [==============================] - 1s 2ms/step - loss: 0.8059 - accuracy: 0.7449 - val_loss: 0.4957 - val_accuracy: 0.8502
Epoch 2/30
191/191 [==============================] - 0s 1ms/step - loss: 0.3823 - accuracy: 0.8729 - val_loss: 0.2804 - val_accuracy: 0.9065
Epoch 3/30
191/191 [==============================] - 0s 1ms/step - loss: 0.2437 - accuracy: 0.9121 - val_loss: 0.1949 - val_accuracy: 0.9281
Epoch 4/30
191/191 [==============================] - 0s 1ms/step - loss: 0.1819 - accuracy: 0.9350 - val_loss: 0.1534 - val_accuracy: 0.9444
Epoch 5/30
191/191 [==============================] - 0s 1ms/step - loss: 0.1407 - accuracy: 0.9566 - val_loss: 0.1216 - val_accuracy: 0.9621
Epoch 6/30
191/191 [==============================] - 0s 1ms/step - loss: 0.1159 - accuracy: 0.9648 - val_loss: 0.1046 - val_accuracy: 0.9725
Epoch 7/30
191/191 [==============================] - 0s 1ms/step - loss: 0.1030 - accuracy: 0.9694 - val_loss: 0.0921 - val_accuracy: 0.9699
Epoch 

In [ ]:
# 6) evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", acc)

Test accuracy: 0.9827315807342529


In [ ]:
# ---- show predicted AQI category on TEST set ----

# predict on test
y_pred = model.predict(X_test)
y_pred_label = le.inverse_transform(y_pred.argmax(axis=1))

# đưa về DataFrame để dễ nhìn
df_test_result = pd.DataFrame({
    "Predicted_AQI_Category": y_pred_label
})

# in ra một số dòng xem kết quả dự đoán
print("Sample prediction on TEST set")
print(df_test_result[["Predicted_AQI_Category"]].head(20))

# nếu muốn xem accuracy nhanh
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")


60/60 [==============================] - 0s 703us/step
Sample prediction on TEST set
   Predicted_AQI_Category
0               Unhealthy
1        Unhealthy for SG
2               Unhealthy
3        Unhealthy for SG
4                Moderate
5                Moderate
6                Moderate
7                Moderate
8                Moderate
9        Unhealthy for SG
10               Moderate
11                   Good
12               Moderate
13              Unhealthy
14              Unhealthy
15              Unhealthy
16       Unhealthy for SG
17       Unhealthy for SG
18               Moderate
19               Moderate

Test Accuracy: 0.9827
